In [1]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

# Instantiates a client
client = language.LanguageServiceClient()


In [2]:
import pandas as pd

df = pd.read_json('data/1000_threads.json')

result = []
counter = 0
for index, row in df.iloc[:,:].iterrows():
    texts = [row['init_post']] + row['comments']
    for text in texts:
        document = types.Document(
                    content=text,
                    type=enums.Document.Type.PLAIN_TEXT,
                    language='en')
        entities = client.analyze_entity_sentiment(document).entities
        result.append(entities)
        

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/arnaudstiegler/miniconda3/envs/forum_scraping/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-8708d2901eb1>", line 14, in <module>
    entities = client.analyze_entity_sentiment(document).entities
  File "/Users/arnaudstiegler/miniconda3/envs/forum_scraping/lib/python3.6/site-packages/google/cloud/language_v1/gapic/language_service_client.py", line 378, in analyze_entity_sentiment
    request, retry=retry, timeout=timeout, metadata=metadata
  File "/Users/arnaudstiegler/miniconda3/envs/forum_scraping/lib/python3.6/site-packages/google/api_core/gapic_v1/method.py", line 143, in __call__
    return wrapped_func(*args, **kwargs)
  File "/Users/arnaudstiegler/miniconda3/envs/forum_scraping/lib/python3.6/site-packages/google/api_core/retry.py", line 273, in retry_wrapped_func
    on_error=on_error,
  File "/Users/arnaudstiegle

KeyboardInterrupt: 

In [ ]:
processed_docword_sentiment = {}
for i in range(len(result)):
    for j in range(len(result[i])):
        if(result[i][j].name not in processed_docword_sentiment.keys()):
            processed_docword_sentiment[result[i][j].name] = {'score':[result[i][j].sentiment.score],
                                                             'magnitude':[result[i][j].sentiment.magnitude]}
        else:
            processed_docword_sentiment[result[i][j].name]['score'].append(result[i][j].sentiment.score)
            processed_docword_sentiment[result[i][j].name]['magnitude'].append(result[i][j].sentiment.magnitude)

In [ ]:
import json

with open('data/docword_sentiment.json', 'w') as outfile:
    json.dump(processed_docword_sentiment, outfile, indent=4, ensure_ascii=False)

In [15]:
# Detects entities in the document. You can also analyze HTML with:
#   document.type == enums.Document.Type.HTML
entities = client.analyze_entities(document).entities

# entity types from enums.Entity.Type
entity_type = ('UNKNOWN', 'PERSON', 'LOCATION', 'ORGANIZATION',
               'EVENT', 'WORK_OF_ART', 'CONSUMER_GOOD', 'OTHER')

for entity in entities:
    print('=' * 20)
    print(u'{:<16}: {}'.format('name', entity.name))
    print(u'{:<16}: {}'.format('type', entity_type[entity.type]))
    print(u'{:<16}: {}'.format('metadata', entity.metadata))
    print(u'{:<16}: {}'.format('salience', entity.salience))
    print(u'{:<16}: {}'.format('wikipedia_url',
          entity.metadata.get('wikipedia_url', '-')))

name            : Obama
type            : PERSON
metadata        : {'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'mid': '/m/02mjmr'}
salience        : 0.8497163653373718
wikipedia_url   : https://en.wikipedia.org/wiki/Barack_Obama
name            : Trump
type            : PERSON
metadata        : {'wikipedia_url': 'https://en.wikipedia.org/wiki/Donald_Trump', 'mid': '/m/0cqt90'}
salience        : 0.09013491868972778
wikipedia_url   : https://en.wikipedia.org/wiki/Donald_Trump
name            : states
type            : LOCATION
metadata        : {}
salience        : 0.060148686170578
wikipedia_url   : -
